## Cost of Equity, Cost of Debt, and WACC

This script uses the yfinance library to get real-time data for Reliance and the Indian market, then applies the formulas for the Cost of Equity, Cost of Debt, and WACC.

📈 Results and Interpretation
Executing the script with live data would yield a result similar to this:

- Risk-Free Rate: The current yield on the 10-year Indian Government Bond. (e.g., 7.00%)

- Beta: A measure of Reliance's volatility relative to the Nifty 50. (e.g., 1.11)

- Market Return: The annualized return of the Nifty 50 over the last 5 years. (e.g., 12.00%)



In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

In [ ]:
# --- 1. Define Tickers and Data Period ---
reliance_ticker = 'RELIANCE.NS'
market_ticker = '^NSEI'

data_period = '5y'
print("Fetching financial data from Yahoo Finance...\n")

# --- 2. Fetch Data and Define Key Inputs ---
try:
    # Get Reliance data
    reliance = yf.Ticker(reliance_ticker)
    reliance_info = reliance.info

    # Get Market (Nifty 50) historical data
    market_data = yf.download(market_ticker, period=data_period, auto_adjust=False)

    # Manual Input for Risk-Free Rate
    risk_free_rate = 0.0645

except Exception as e:
    print(f"An error occurred while fetching data: {e}")
    print("Execution will continue with available data, but results may be incomplete.")
    # Set default values if an error occurs to prevent NameError
    reliance_info = {}
    market_data = pd.DataFrame()
    risk_free_rate = 0.07 # Default value

# --- 3. Calculate Cost of Equity (CAPM) ---
print("Calculating Cost of Equity (CAPM)...")
beta = reliance_info.get('beta', None)
if beta is None:
    print("Beta not found. Cannot calculate Cost of Equity.")
    cost_of_equity = None
else:
    market_returns = market_data['Adj Close'].pct_change().dropna()
    market_return_annual = ((1 + market_returns.mean())**252 - 1).iloc[0]

    print(f"  Risk-Free Rate (10Y Yield): {risk_free_rate:.2%}")
    print(f"  Beta for Reliance: {beta:.2f}")
    print(f"  Annualized Market Return (Nifty 50): {market_return_annual:.2%}\n")

    cost_of_equity = risk_free_rate + beta * (market_return_annual - risk_free_rate)
    print(f"✅ Calculated Cost of Equity (Ke): {cost_of_equity:.2%}\n")

# --- 4. Calculate Cost of Debt (After-Tax) ---
print("Calculating Cost of Debt...")
interest_rate_on_debt = 0.0865
tax_rate = reliance_info.get('effectiveTaxRate', None)
if tax_rate is None:
    tax_rate = 0.2517
    print("  Effective tax rate not found. Using a standard corporate tax rate of 25.17%.")

cost_of_debt = interest_rate_on_debt * (1 - tax_rate)
print(f"  Hypothetical Interest Rate: {interest_rate_on_debt:.2%}")
print(f"  Effective Tax Rate: {tax_rate:.2%}\n")
print(f"✅ Calculated Cost of Debt (Kd): {cost_of_debt:.2%}\n")

# --- 5. Calculate WACC ---
print("Calculating Weighted Average Cost of Capital (WACC)...")
market_cap_equity = reliance_info.get('marketCap')
total_debt = reliance_info.get('totalDebt')

if cost_of_equity is None or market_cap_equity is None or total_debt is None:
    print("One or more key variables not found. Cannot calculate WACC.")
else:
    total_value = market_cap_equity + total_debt
    weight_equity = market_cap_equity / total_value
    weight_debt = total_debt / total_value

    print(f"  Market Value of Equity (₹): {market_cap_equity:,}")
    print(f"  Market Value of Debt (₹): {total_debt:,}")
    print(f"  Weight of Equity: {weight_equity:.2%}")
    print(f"  Weight of Debt: {weight_debt:.2%}\n")

    wacc = (weight_equity * cost_of_equity) + (weight_debt * cost_of_debt)
    print(f"🎉 Final WACC for Reliance Industries: {wacc:.2%}")

Fetching financial data from Yahoo Finance...



[*********************100%***********************]  1 of 1 completed

Calculating Cost of Equity (CAPM)...
  Risk-Free Rate (10Y Yield): 6.45%
  Beta for Reliance: 0.22
  Annualized Market Return (Nifty 50): 18.73%

✅ Calculated Cost of Equity (Ke): 9.11%

Calculating Cost of Debt...
  Effective tax rate not found. Using a standard corporate tax rate of 25.17%.
  Hypothetical Interest Rate: 8.65%
  Effective Tax Rate: 25.17%

✅ Calculated Cost of Debt (Kd): 6.47%

Calculating Weighted Average Cost of Capital (WACC)...
  Market Value of Equity (₹): 18,623,424,888,832
  Market Value of Debt (₹): 3,695,749,890,048
  Weight of Equity: 83.44%
  Weight of Debt: 16.56%

🎉 Final WACC for Reliance Industries: 8.68%


💰 Equity Valuation with the Dividend Discount Model (DDM)
The Dividend Discount Model (DDM) is a method of valuing a stock by projecting the value of all future dividends and discounting them back to their present value. The most common form of DDM is the Gordon Growth Model, which assumes a company's dividends will grow at a constant rate indefinitely.

We will implement this in Python by first calculating the historical dividend growth rate to use as a proxy for g.

🐍 Python Implementation
This script fetches Reliance's historical dividend data from Yahoo Finance, calculates the compound annual growth rate (CAGR), and then uses the Gordon Growth Model to derive an intrinsic value.

In [ ]:
# --- 1. Define Ticker and Inputs ---
reliance_ticker = 'RELIANCE.NS'

# Using the previously calculated Cost of Equity (Ke)
cost_of_equity = 0.1255  # Or use a new value based on your analysis

# --- 2. Fetch Data and Calculate Growth Rate (g) ---
print("Fetching historical dividend data from Yahoo Finance...\n")

try:
    # Get Reliance historical dividends
    dividends = yf.Ticker(reliance_ticker).dividends

    if dividends.empty:
        print("No dividend data found for Reliance. DDM is not applicable.")
        exit()

    # Get the first and last dividend payments
    first_dividend = dividends.iloc[0]
    last_dividend = dividends.iloc[-1]

    # Calculate the number of years over which the dividends were paid
    # Use 252 as an approximation for trading days per year
    num_years = (dividends.index[-1] - dividends.index[0]).days / 365.25

    # Calculate the Compound Annual Growth Rate (CAGR)
    dividend_growth_rate = (last_dividend / first_dividend)**(1/num_years) - 1

    # Check for negative or unrealistic growth rate
    if dividend_growth_rate <= 0 or dividend_growth_rate >= cost_of_equity:
        print(f"Calculated dividend growth rate ({dividend_growth_rate:.2%}) is unsuitable for the Gordon Growth Model.")
        print("The model requires g < Ke. Please use a different valuation method or a more reasonable growth rate.")
        exit()

except Exception as e:
    print(f"An error occurred while fetching dividend data: {e}")
    exit()

# --- 3. Perform the Valuation ---
print("Calculating Intrinsic Value using the Gordon Growth Model...\n")

# Get the latest dividend
latest_dividend = dividends.iloc[-1]

# Calculate the expected dividend next year (D1)
D1 = latest_dividend * (1 + dividend_growth_rate)

# Gordon Growth Model formula: Value = D1 / (Ke - g)
intrinsic_value = D1 / (cost_of_equity - dividend_growth_rate)

# --- 4. Display Results ---
print(f"  Latest Annual Dividend Per Share (DPS): ₹{latest_dividend:.2f}")
print(f"  Calculated Dividend Growth Rate (g): {dividend_growth_rate:.2%}")
print(f"  Assumed Cost of Equity (Ke): {cost_of_equity:.2%}\n")

print(f"🎉 **Intrinsic Value per Share:** ₹{intrinsic_value:.2f}")

Fetching historical dividend data from Yahoo Finance...

Calculated dividend growth rate (9.66%) is unsuitable for the Gordon Growth Model.
The model requires g < Ke. Please use a different valuation method or a more reasonable growth rate.
Calculating Intrinsic Value using the Gordon Growth Model...

  Latest Annual Dividend Per Share (DPS): ₹5.50
  Calculated Dividend Growth Rate (g): 9.66%
  Assumed Cost of Equity (Ke): 9.11%

🎉 **Intrinsic Value per Share:** ₹-1104.28


- The model needs the cost to be higher than the growth rate, else it will retrun negative value

In [ ]:
# --- 1. Define Ticker and Key Inputs ---
reliance_ticker = 'RELIANCE.NS'

# Using the previously calculated WACC as the discount rate
wacc = 0.1152

# Manual inputs for growth assumptions (These are critical assumptions!)
# We'll assume a high-growth phase followed by a stable, long-term growth phase.
high_growth_years = 5  # Length of the high-growth phase
# Projected annual revenue growth rates for the high-growth phase
revenue_growth_rates = [0.10, 0.08, 0.07, 0.06, 0.05]
# Long-term stable growth rate for the terminal value calculation (g)
perpetual_growth_rate = 0.03  # A common proxy is the long-term GDP growth rate

print("Setting up the Discounted Cash Flow (DCF) model...\n")

# --- 2. Fetch Financial Data ---
try:
    reliance_info = yf.Ticker(reliance_ticker).info
    # Get the latest revenue and NOPAT (Net Operating Profit After Tax)
    # yfinance often lacks detailed NOPAT, so we'll use a simplified approach
    # and a proxy for tax rate.
    latest_revenue = reliance_info.get('totalRevenue', None)
    latest_ebit = reliance_info.get('ebitda', None) # Use EBITDA as a proxy for EBIT
    latest_depreciation = reliance_info.get('depreciation', None) # Not always available
    tax_rate = reliance_info.get('effectiveTaxRate', 0.2517)

    if latest_revenue is None or latest_ebit is None:
        print("Required financial data (Revenue or EBIT) not found. Cannot proceed.")
        exit()

except Exception as e:
    print(f"An error occurred while fetching data: {e}")
    exit()

# --- 3. Project Free Cash Flows to the Firm (FCFF) ---
print("Projecting Free Cash Flows...\n")
projected_fcff = []
current_revenue = latest_revenue

# To simplify, we'll assume a constant NOPAT margin for the forecast period
# This is a key simplifying assumption; a real model would project this.
# NOPAT Margin = (EBITDA - D&A) * (1-t) / Revenue
# We'll use a conservative NOPAT margin based on recent financials.
nopat_margin = (latest_ebit - (latest_depreciation if latest_depreciation else 0)) / latest_revenue * (1-tax_rate)
# Assumed NWC change and CAPEX as a percentage of revenue
nwc_change_rate = 0.01
capex_rate = 0.03

for i in range(high_growth_years):
    # Project revenue
    current_revenue = current_revenue * (1 + revenue_growth_rates[i])

    # Calculate NOPAT
    nopat = current_revenue * nopat_margin

    # Calculate FCFF = NOPAT - Change in NWC - CAPEX
    # We'll assume NWC changes and CAPEX as a percentage of the new revenue
    change_nwc = current_revenue * nwc_change_rate
    capex = current_revenue * capex_rate
    fcff = nopat - change_nwc - capex
    projected_fcff.append(fcff)

    print(f"  Year {i+1} Projected FCFF: ₹{fcff:,.0f}")

# --- 4. Calculate Terminal Value ---
# The Terminal Value captures the value of all cash flows beyond the forecast period.
# TV = FCFF_last_year * (1 + g) / (WACC - g)
print("\nCalculating Terminal Value...")
final_year_fcff = projected_fcff[-1]
terminal_value = (final_year_fcff * (1 + perpetual_growth_rate)) / (wacc - perpetual_growth_rate)
print(f"  Terminal Value: ₹{terminal_value:,.0f}\n")

# --- 5. Discount All Cash Flows and Terminal Value to Present Value ---
print("Discounting all values to Present Value (PV)...")
# Calculate the present value of each projected FCFF
present_value_of_fcff = sum(fcff / (1 + wacc)**(i+1) for i, fcff in enumerate(projected_fcff))

# Calculate the present value of the Terminal Value
present_value_of_terminal_value = terminal_value / (1 + wacc)**high_growth_years

# Total Enterprise Value (EV) = PV of FCFFs + PV of Terminal Value
enterprise_value = present_value_of_fcff + present_value_of_terminal_value
print(f"  Total Enterprise Value (EV): ₹{enterprise_value:,.0f}\n")

# --- 6. Calculate Equity Value per Share ---
# Equity Value = EV - Total Debt + Cash and Equivalents
total_debt = reliance_info.get('totalDebt')
cash_and_equivalents = reliance_info.get('totalCash')
shares_outstanding = reliance_info.get('sharesOutstanding')

if total_debt is None or cash_and_equivalents is None or shares_outstanding is None:
    print("Debt, cash, or shares outstanding data not found. Cannot calculate final value per share.")
    exit()

equity_value = enterprise_value - total_debt + cash_and_equivalents
intrinsic_value_per_share = equity_value / shares_outstanding

print(f"  Total Enterprise Value: ₹{enterprise_value:,.0f}")
print(f"  Total Debt: ₹{total_debt:,.0f}")
print(f"  Cash & Equivalents: ₹{cash_and_equivalents:,.0f}")
print(f"  Shares Outstanding: {shares_outstanding:,}")
print("\n🎉 **Intrinsic Value per Share (DCF):** ₹{:.2f}".format(intrinsic_value_per_share))

Setting up the Discounted Cash Flow (DCF) model...

Projecting Free Cash Flows...

  Year 1 Projected FCFF: ₹979,467,054,041
  Year 2 Projected FCFF: ₹1,057,824,418,364
  Year 3 Projected FCFF: ₹1,131,872,127,650
  Year 4 Projected FCFF: ₹1,199,784,455,309
  Year 5 Projected FCFF: ₹1,259,773,678,074

Calculating Terminal Value...
  Terminal Value: ₹15,229,658,314,747

Discounting all values to Present Value (PV)...
  Total Enterprise Value (EV): ₹12,880,291,206,534

  Total Enterprise Value: ₹12,880,291,206,534
  Total Debt: ₹3,695,749,890,048
  Cash & Equivalents: ₹2,252,110,036,992
  Shares Outstanding: 13,532,499,968

🎉 **Intrinsic Value per Share (DCF):** ₹845.12


Firm value, also known as Enterprise Value (EV), is the sum of the discounted present values of all future Free Cash Flows to the Firm (FCFF) plus the Terminal Value. The python script below provides the complete code for this calculation.

The script first projects the cash flows, then discounts them to get the firm value, and finally derives the equity value per share from the firm value.

In [ ]:
# --- 1. Define Ticker and Key Inputs ---
reliance_ticker = 'RELIANCE.NS'

# Using the previously calculated WACC as the discount rate
wacc = 0.1152

# Manual inputs for growth assumptions (These are critical assumptions!)
high_growth_years = 5  # Length of the high-growth phase
revenue_growth_rates = [0.10, 0.08, 0.07, 0.06, 0.05]
perpetual_growth_rate = 0.03  # Long-term stable growth rate for the terminal value

print("Setting up the Discounted Cash Flow (DCF) model...\n")

# --- 2. Fetch Financial Data ---
try:
    reliance_info = yf.Ticker(reliance_ticker).info
    latest_revenue = reliance_info.get('totalRevenue', None)
    latest_ebit = reliance_info.get('ebitda', None)
    latest_depreciation = reliance_info.get('depreciation', None)
    tax_rate = reliance_info.get('effectiveTaxRate', 0.2517)

    if latest_revenue is None or latest_ebit is None:
        print("Required financial data (Revenue or EBIT) not found. Cannot proceed.")
        exit()

except Exception as e:
    print(f"An error occurred while fetching data: {e}")
    exit()

# --- 3. Project Free Cash Flows to the Firm (FCFF) ---
print("Projecting Free Cash Flows...\n")
projected_fcff = []
current_revenue = latest_revenue
nopat_margin = (latest_ebit - (latest_depreciation if latest_depreciation else 0)) / latest_revenue * (1-tax_rate)
nwc_change_rate = 0.01
capex_rate = 0.03

for i in range(high_growth_years):
    current_revenue = current_revenue * (1 + revenue_growth_rates[i])
    nopat = current_revenue * nopat_margin
    change_nwc = current_revenue * nwc_change_rate
    capex = current_revenue * capex_rate
    fcff = nopat - change_nwc - capex
    projected_fcff.append(fcff)

    print(f"  Year {i+1} Projected FCFF: ₹{fcff:,.0f}")

# --- 4. Calculate Terminal Value ---
print("\nCalculating Terminal Value...")
final_year_fcff = projected_fcff[-1]
terminal_value = (final_year_fcff * (1 + perpetual_growth_rate)) / (wacc - perpetual_growth_rate)
print(f"  Terminal Value: ₹{terminal_value:,.0f}\n")

# --- 5. Discount All Cash Flows and Terminal Value to Present Value ---
print("Discounting all values to Present Value (PV)...")
present_value_of_fcff = sum(fcff / (1 + wacc)**(i+1) for i, fcff in enumerate(projected_fcff))
present_value_of_terminal_value = terminal_value / (1 + wacc)**high_growth_years

# Total Enterprise Value (EV) = PV of FCFFs + PV of Terminal Value
enterprise_value = present_value_of_fcff + present_value_of_terminal_value
print(f"  Total Enterprise Value (EV): ₹{enterprise_value:,.0f}\n")

# --- 6. Calculate Equity Value per Share ---
total_debt = reliance_info.get('totalDebt')
cash_and_equivalents = reliance_info.get('totalCash')
shares_outstanding = reliance_info.get('sharesOutstanding')

if total_debt is None or cash_and_equivalents is None or shares_outstanding is None:
    print("Debt, cash, or shares outstanding data not found. Cannot calculate final value per share.")
    exit()

equity_value = enterprise_value - total_debt + cash_and_equivalents
intrinsic_value_per_share = equity_value / shares_outstanding

print(f"  Total Enterprise Value: ₹{enterprise_value:,.0f}")
print(f"  Total Debt: ₹{total_debt:,.0f}")
print(f"  Cash & Equivalents: ₹{cash_and_equivalents:,.0f}")
print(f"  Shares Outstanding: {shares_outstanding:,}")
print("\n🎉 **Intrinsic Value per Share (DCF):** ₹{:.2f}".format(intrinsic_value_per_share))

Setting up the Discounted Cash Flow (DCF) model...

Projecting Free Cash Flows...

  Year 1 Projected FCFF: ₹979,467,054,041
  Year 2 Projected FCFF: ₹1,057,824,418,364
  Year 3 Projected FCFF: ₹1,131,872,127,650
  Year 4 Projected FCFF: ₹1,199,784,455,309
  Year 5 Projected FCFF: ₹1,259,773,678,074

Calculating Terminal Value...
  Terminal Value: ₹15,229,658,314,747

Discounting all values to Present Value (PV)...
  Total Enterprise Value (EV): ₹12,880,291,206,534

  Total Enterprise Value: ₹12,880,291,206,534
  Total Debt: ₹3,695,749,890,048
  Cash & Equivalents: ₹2,252,110,036,992
  Shares Outstanding: 13,532,499,968

🎉 **Intrinsic Value per Share (DCF):** ₹845.12


💰 Cash Flow-Based Equity Valuation (DCF Model)
A cash flow-based equity valuation, commonly known as a Discounted Cash Flow (DCF) model, is a powerful method for valuing a company. Unlike the Dividend Discount Model (DDM), the DCF model values a company based on its projected free cash flows to the firm (FCFF), which represents the cash generated before any debt payments but after all capital expenditures.

The DCF valuation process involves three main steps:

Project Free Cash Flows (FCFF): Estimate the company's FCFF for a specific forecast period (e.g., 5-10 years).

Calculate Terminal Value: Estimate the value of the company's cash flows beyond the forecast period, assuming a stable growth rate into perpetuity.

Discount to Present Value: Discount all projected FCFFs and the Terminal Value back to their present value using the Weighted Average Cost of Capital (WACC).

For equity valuation, you can use either Free Cash Flow to the Firm (FCFF) or Free Cash Flow to Equity (FCFE), but they require different discount rates and lead to the same result if done correctly.

FCFF (Free Cash Flow to the Firm): This is the cash flow available to all capital providers (both debt and equity holders) after paying all operating expenses and making necessary investments. Since FCFF represents cash flow to the entire firm, it must be discounted using the Weighted Average Cost of Capital (WACC). The result of this valuation is the Enterprise Value (EV) of the firm. To find the equity value, you must subtract the market value of debt and add back cash and cash equivalents.


FCFE (Free Cash Flow to Equity): This is the cash flow available only to equity holders after all operating expenses, capital investments, and payments to debt holders (interest and principal) have been made. Since FCFE represents cash flow to equity holders, it must be discounted using the Cost of Equity (K
e
​
 ). The result of this valuation is the Equity Value directly.

In practice, both methods should yield the same equity value. Most analysts prefer using FCFF because it is independent of the company's capital structure. This makes it easier to compare the value of different companies and less sensitive to changes in debt levels.

The choice of which cash flow to use is crucial and depends on what you are trying to value and the discount rate you have.

To value the entire firm (Enterprise Value), use FCFF and WACC.

To value just the equity (Equity Value), use FCFE and the Cost of Equity.

This script provides a basic DCF model for Reliance Industries. It requires manual input for projected growth rates and margins as these are strategic assumptions that cannot be reliably pulled from public data.